In [1]:
%load_ext sql

In [2]:
%sql sqlite:///lab3.db3

'Connected: @lab3.db3'

Displays all Tables

In [3]:
%%sql sqlite://

SELECT 
    name
FROM 
    sqlite_master
WHERE 
    type = 'table' AND
    name NOT LIKE 'sqlite_%'

Done.


name
students
courses
scores
teachers


lab3.db3 has four tables:

students (sno, sname, sgender, sbirthday, class)

courses (cno, cname, tno)

scores (sno, cno, grade)

teachers (tno, tname, tgender, tbirthday, title, department)

Question 1:

In the score table, find the student number that has all the grades in between 90 and 70.

Answer 1:

1. Goal: We want the student number. This is in students and scores
2. Constraints: MAX(grade) < 90, MIN (grade) > 70. grade is in scores
3. We will use HAVING

In [4]:
%%sql sqlite://

SELECT 
    sno
FROM 
    scores
GROUP BY 
    sno
HAVING 
    MAX(grade) <= 90 AND MIN(grade >= 70)

Done.


sno
105
108


Question 2:

For all the courses that were taken by class 15033, calculate the average grade.

Answer 2:

1. Goal: We want an average grade. This comes from scores.
2. Constraint: We only want the average grade of courses by class 15033. The class number can be found in students.
3. We can either join students and scores, or run a nested query.

In [5]:
%%sql sqlite://
    
SELECT
    AVG (scores.grade)
FROM
    scores
WHERE
    scores.sno IN
    (
        SELECT
            students.sno
        FROM
            students
        WHERE
            students.class = 15033
    )

Done.


AVG (scores.grade)
79.66666666666667


Question 3:

Find the class number that has at least two male students.

Answer 3:

1. Goal: We want a class number. This is found in students.
2. Constraint: We want class numbers with at least two males. Student gender can be found in students.

In [6]:
%%sql sqlite://
    
SELECT
     students.class
FROM 
    students
GROUP BY
    students.class
HAVING
    COUNT(students.sgender = 'male') >= 2
    
    

Done.


class
15031
15033


Question 4:

Find the teacher's name in CS and EE department, where they have different title. Return both name and title.

Answer 4:

1. Goal: We want teacher names. They can be found in teachers.
2. Constraints: We want teacher names in the CS and EE departments. More specifically, we want them to have different titles in each. These can also be found in teachers.

In [7]:
%%sql sqlite://

SELECT
    t1.tname
FROM
    teachers AS t1
WHERE
    t1.department = 'CS' AND
    t1.tname IN
    (
    SELECT
        t2.tname
    FROM
        teachers AS t2
    WHERE
        t2.department = 'EE' AND
        t1.title <> t2.title
    )

Done.


tname


Question 5

Find the students, who took the course number "3-105" and have earned a grade, at least, higher than the
students who took "3-245" course. Return the results in a descending order of grade.

Answer 5

1. Goal: We want to find student numbers. This can be found in students or scores.
2. Constraints: We want to find students who took course "3-105." The course number can be found in scores and courses. More specifically, we want students who took "3-105" who had a higher grade than the grades of students who took course "3-245."

In [26]:
%%sql sqlite://

SELECT 
    DISTINCT scores.sno
FROM
    scores
WHERE
    scores.sno IN
    (
    SELECT
        scores.sno
    FROM
        scores
    WHERE
        scores.cno = '3-105'
    )
GROUP BY 
    grade
HAVING
    MAX(grade) > 
    (
        SELECT 
            MAX(scores.grade)
         FROM
            scores
         WHERE 
            scores.cno = '3-245'
    )
ORDER BY 
    scores.grade DESC

Done.


sno
103
107
105


Question 6:

Find the students who took more than one course, and return the students' names that is not the one with the highest grade.

Answer 6:

1. Goal: We want to find student numbers. This can be found in scores.
2. Constraint: We only want to find the numbers of students who took more than one course. This can also be found in scores. More specifically, we want to omit the student with the highest score.

Note: I wrote the same subquery three times because when I renamed it with AS it wasn't recognized when I called it

In [73]:
%%sql sqlite://
    
SELECT
    DISTINCT scores.sno
FROM
    scores
WHERE
    sno IN
    (
        SELECT
            sno
        FROM
            scores
        GROUP BY
            scores.sno
        HAVING 
            COUNT (scores.cno) > 1
    ) AND
    sno NOT IN
    (
        SELECT
            sno
        FROM
        (
            SELECT
                *
            FROM
                scores
            GROUP BY
                scores.sno
            HAVING
                COUNT (scores.cno) > 1
        ) AS multi_class_scores
        GROUP BY
            grade
        HAVING
            grade = (                          
                        SELECT                 
                            MAX(grade)
                        FROM 
                        (
                            SELECT
                                *
                            FROM
                                scores
                            GROUP BY
                                scores.sno
                            HAVING
                                COUNT (scores.cno) > 1
                        )
                    )
    )    

Done.


sno
101
105
107
108
109
